In [1]:
# -*- coding: utf-8 -*-
"""
Created on Aug 30 10:45:00 2018
@author: Fernanda Farinelli
@description: 

This program connect to NYT (New York Times) database, read the caratulas collection:
1) Find documents that has the word "feminicide" or "femicide" in title (headline field)


"""

'\nCreated on Aug 30 10:45:00 2018\n@author: Fernanda Farinelli\n@description: \n\nThis program connect to NYT (New York Times) database, read the caratulas collection:\n1) Find documents that has the word "feminicide" or "femicide" in title (headline field)\n\n\n'

In [2]:
# Defining constants
server = 'localhost'
port = 27017
dbname = 'nyt'
collectionname = 'caratulas'

In [3]:
# Defining functions

def getServerConnection():
    global server, port
    #Creating a connection to database server
    return pymongo.MongoClient(server,port)

#Defining a database in such server
def getDBConnection(con):
    global dbname
    #bancoDados = conexao['nyt']
    #bancoDados = conexao.nyt    
    return con[dbname]

def getCollection(dbs):
    global collectionname
    #Connecting to a database collection
    #colecao = bancoDados["caratulas"]
    return dbs[collectionname]

def getDbCollection(con):
    global dbname, collectionname
    return con[dbname][collectionname]

def searchWord(collection,field,regex):
    print('Start:',datetime.datetime.now())
    r = collection.aggregate([
        {
               "$match": {field.lower(): {'$regex': regex}}
        },
        {
                "$group": {"_id": "$tags", "count": {"$sum": 1}}
        },
        {
            "$group": {"_id": None, "total": {"$sum": 1}, "details":{"$push":{"groupby": "$_id", "count": "$count"}}}
        },
        {
                "$sort": SON([("count", -1), ("_id", -1)])
        }
        ])
    print('End:',datetime.datetime.now())
    print()    
    return r

In [12]:
# Import libraries

import pymongo
import datetime
from pprint import *
from bson.son import SON

#import nltk

print ('Mongo version', pymongo.__version__)
print()

Mongo version 3.7.0



In [5]:
#Getting a database server connection
connection = getServerConnection()

#Getting a database
database = getDBConnection(connection)

#Getting a collection
collection = getCollection(database)

#Counting total of documents in such collection
print("Total de registros banco de dados NYT:",collection.estimated_document_count())
print()

Total de registros banco de dados NYT: 14099370



In [11]:
import re
from pprint import *

regx = re.compile(".Topless.*", re.IGNORECASE)
#db.users.find_one({"files": regx})

#printFormat = pprint.PrettyPrinter(indent=4)

#printFormat.pprint(collection.find_one({"headline.main":regx}))


pprint(collection.find_one({"headline.main":regx}))

print()

{'_id': '4fd2135b8eb7c8105d78a7a6',
 'abstract': None,
 'blog': [],
 'byline': {'original': 'By RICHARD PEREZ-PENA',
            'person': [{'firstname': 'Richard',
                        'lastname': 'PEREZ-PENA',
                        'organization': '',
                        'rank': 1,
                        'role': 'reported'}]},
 'document_type': 'article',
 'headline': {'main': 'City Loses Another Round In Fight With Topless Bars'},
 'keywords': [],
 'lead_paragraph': "For the second time in five months, the state's highest "
                   'court today said the Giuliani administration was '
                   'overzealous in enforcing a zoning law that restricted '
                   'sex-related businesses, this time batting down a city '
                   'attempt to close a topless bar in Chelsea. The Court of '
                   'Appeals found unanimously that the VIP Club at 20 West '
                   '20th Street had successfully skirted the rules on X-rated '

In [14]:
#word = '/^Yeltsin/'

pprint(collection.find_one({"headline.main":{'$regex': '.*topless.*i'}}))

None


In [ ]:
#Looking for *femicide.*" or ".*feminicide.*" 
print('Start:',datetime.datetime.now())
result = collection.aggregate([
        {
               #"$match": {"headline.main".lower(): {'$regex': ".*femicide.*" or ".*feminicide.*"}}
                "$match": {"headline.main".lower(): {'$regex': ".*topless.*"}}
        },
        {
                "$group": {"_id": "$tags", "count": {"$sum": 1}}
        },
        {
            "$group": {"_id": None, "total": {"$sum": 1}, "details":{"$push":{"groupby": "$_id", "count": "$count"}}}
        },
        {
                "$sort": SON([("count", -1), ("_id", -1)])
        }
        ])
   
print('End:',datetime.datetime.now())
print()


Start: 2018-09-03 09:29:33.950893


In [ ]:
#print(type(result))
pprint(list(result))

In [15]:
regex = '".*femicide.*" or ".*feminicide.*"'
#regex = '".*federal.*"'
field = "headline.main"

result = searchWord(collection,field,regex)
pprint(list(result))

Start: 2018-09-03 10:29:09.738417
End: 2018-09-03 10:33:17.730601

[]


In [16]:
regex = '".*femicide.*"'
#regex = '".*federal.*"'
field = "abstract"

result = searchWord(collection,field,regex)
pprint(list(result))

Start: 2018-09-03 10:33:17.829607
End: 2018-09-03 10:35:53.290499

[]


In [ ]:
field = "snippet"

result = searchWord(collection,field,regex)
print(list(result))

In [ ]:
field = "lead_paragraph"

result = searchWord(collection,field,regex)
print(list(result))